## Przedstawienie wyników różnych algorytmów poleceń dla wybranego użytkownika

In [1]:
# Jeśli będą porzebne to można odkomentować

# # imports 
# import pandas as pd
# movies = pd.read_csv('data/movie.csv')
# ratings = pd.read_csv('data/rating.csv')

In [2]:
# wybór użytkownika, dla którego będziemy polecać filmy, oraz ile filmów chcemy wyświetlić
user_id = 23
number_of_movies = 10

## Metoda klasteryzacji algorytmem k-średnich

In [3]:
# Czas wykonania tej komórki u mnie na komputerze to około minuty
from clustering import find_movies

result = find_movies(number_of_movies, user_id)

print(result)

Termination criterion satisfied
                                               title
0  Raiders of the Lost Ark (Indiana Jones and the...
1                              Godfather, The (1972)
2                            Schindler's List (1993)
3  Lord of the Rings: The Fellowship of the Ring,...
4      Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
5                                 Rear Window (1954)
6                                  Casablanca (1942)
7                              Paths of Glory (1957)
8                     Godfather: Part II, The (1974)
9                                12 Angry Men (1957)


## Metoda 2

In [4]:
# kod

## Metoda 3

In [5]:
# kod

## Porównanie zwróconych wyników

In [6]:
# Sprawdzenie gatunków polecanych filmów 
